# 参考文献

- Deep Learning for Matching in Search and Recommendation
- https://zhuanlan.zhihu.com/p/101136699 推荐系统中的深度匹配模型
- https://zhuanlan.zhihu.com/p/267263561  推荐系统总结之深度召回模型（上）

![推荐和搜索的本质，都是match的过程](https://pic3.zhimg.com/80/v2-18e28b76d97f4421ca943d38270e801e_720w.jpg)

推荐系统和搜索应该是机器学习乃至深度学习在工业界落地应用最多也最容易变现的场景。而无论是搜索还是推荐，本质其实都是匹配，搜索的本质是给定query，匹配doc；推荐的本质是给定user，推荐item。本文主要讲推荐系统里的匹配问题，包括传统匹配模型和深度学习模型。

匹配（matching）是衡量用户对物品的兴趣的过程，也是推荐召回中的工作内容。机器学习中是以learning to match的方式根据输入表示和标记数据学习一个匹配函数。而深度学习在其发展过程中以强大的表示学习和泛化能力加上算力提升、数据规模暴涨都使得深度模型在推荐召回中大放异彩。


# 0 推荐系统概述

## 0.1 推荐系统的本质

定义：系统根据**用户的属性**（如性别、年龄、学历、地域、职业），用户在系统里**过去的行为**（例如浏览、点击、搜索、购买、收藏等），以及**当前上下文环境**（如网络、手机设备、时间等），从而给用户**推荐用户可能感兴趣的物品**（如电商的商品、feeds推荐的新闻、应用商店推荐的app等），从这个过程来看，推荐系统就是一个**给user匹配(match)感兴趣的item**的过程。

## 0.2 推荐和搜索的比较

都是一个match过程，图1.1展示的是一个搜索引擎的架构，需要match的是query和相关的doc；图1.2展示的是一个推荐系统的架构，需要match的是user(可能会带有主动意图的query)和相关的item。

![图1搜素引擎架构](https://pic1.zhimg.com/80/v2-0be4057ee616ff2acd10c92c82336694_720w.jpg)

![图2推荐系统架构](https://pic2.zhimg.com/80/v2-4f04a7013c8af1a71e285a994599b201_720w.jpg)



# 目录

- [1. feature_based的深度模型](#1-feature_based的深度模型)
    - [1.1 FM](#11-FM)

    

# 1. feature_based的深度模型

## 1.1 FM

![图1.1 FM模型的稀疏one-hot特征输入](https://pic1.zhimg.com/80/v2-5baaefa316db3345788b0256a4561134_720w.jpg)

图1.1 FM模型的稀疏one-hot特征输入

![图1.2 FM模型的数学表达](https://pic3.zhimg.com/80/v2-04cca7447d2205adb3edb26a7e1b980a_720w.jpg)

图1.2 FM模型的数学表达

对于每个输入特征，模型都需要学习一个低维的隐向量表示$v$，也就是NN中的embedding层。FM由两部分组成：蓝色的是LR线性模型，红色的是二阶特征组合——两两特征组合，但不和自己组合，向量之间的交叉用向量内积表示。

```python
feats = features['feats']  # [batch_size*feat_num, feat_val_num]
emb = self.emb_module(feats)  # (bs*feature_num, feat_val_num, emb_dim) feat_val_num=1
emb = tf.reshape(emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num, emb_dim) feat_val_num=1 

# -----LR-----
linear_emb = self.linear_emb_module(feats)  # (bs*feature_num, feat_val_num, emb_size) feat_val_num=1,emb_size=1
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1]) # （bs,feat_num）
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True)  # [bs, 1]

# -----二阶-----
# The shape of embedding is [bs, feature_num, emb_dim]
summed_features_emb_square = tf.square(tf.reduce_sum(emb,1))  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
squared_sum_features_emb = tf.reduce_sum(tf.square(emb),1)  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
FM2 = 0.5 * tf.subtract(summed_features_emb_square, squared_sum_features_emb)  # [bs, emb_dim] 
# [bs, emb_dim]的FM结果与特征两两逐元素相乘加和再tf.reduce_sum(pairwise_interaction, 1)消除第二维度（feat_num*(feat_num-1)/2）得到的[bs, emb_dim]结果一致

FM = tf.reduce_sum(FM2, 1) + linear_out # [bs,] 按照FM计算公式有此步骤

```

## 1.2 Wide&Deep

-  [2016]"Wide & Deep Learning for Recommender Systems": https://arxiv.org/abs/1606.07792 [2] [official] https://github.com/tensorflow/models/tree/master/official/wide_deep 


![wd模型图](https://github.com/hsux/Distributed-training/raw/master/images/wd_imgs/wd.jpg)

（1） 记忆性：wide部分长处在于学习样本中的**高频**部分。
- 优点是模型的**记忆性好**，对于样本中出现过的**高频低阶特征能够用少量参数学习**；
- 缺点是模型的**泛化能力差**，例如对于没有见过的ID类特征，模型学习能力较差。

（2） 泛化性：deep部分长处在于学习样本中的**长尾**部分。
- 优点是**泛化能力强**，对于**少量出现过的样本甚至没有出现过的样本都能做出预测（非零的embedding向量）**，容易带来惊喜。
- 缺点是模型**对于低阶特征的学习需要用较多参数才能等同wide部分效果**，而且泛化能力强某种程度上也**可能导致过拟合出现bad case**。尤其对于冷启动的一些item，也有可能给用户带来惊吓。

![模型框架2](https://pic1.zhimg.com/80/v2-0bb7650022db2d677b029bb39537ed60_720w.jpg)

虽然模型的deep部分拟合和泛化能力很强，但绝对不意味着把特征交叉都交给MLP就够了。实际证明，对于重要的一些人工经验的特征，对于提升整体效果还是非常重要的，如上图所示。这个人工特征的所谓缺点，也是后续各种模型结构想对其进行“自动化”的优化点。


## 1.3 Deep Crossing

- 2016 微软

wide&deep的MLP部分是全连接网络，每层的输入是前一层的输出，受限于模型结构，**越往后的层越难学习到原始输入的表达，一般深度不会太深**，超过5层的网络在工业界已经算很少见了。为了解决这个问题，deep crossing网络引入了resnet残差网络的概念，**通过`short-cut`，在MLP的深层网络，也能接收来自第一层的输入，这样可以使得模型的深度达到10层之多**。

![deep crossing模型框架](https://pic4.zhimg.com/80/v2-cc6b972eb9e73816a454d154413a4a73_720w.jpg)

上述提到的wide&deep以及deep crossing框架更像是在模型结构做的改进，一个引入了wide&deep，一个引入了resnet，**特征层面并没有做太多改造**，如何体现feature-base呢？sigIR2017就有一篇文章做了个实验，对wide&deep以及Deep&Cross实验*按照embedding是否做初始化分别做了实验*。实验发现，如果embedding是随机初始化的，两个深度模型连基础的FM模型都打不过；哪怕经过FM初始化了embedding，wide&deep模型效果也仅仅略好于FM模型，而deep crossing模型依然比不过FM模型，实验结果如下图所示：

![不同初始化对模型影响](https://pic3.zhimg.com/80/v2-07bbaf8ac3eafe8a87ba09b50f39db26_720w.jpg)

全连接网络表面上看对所有节点都进行了连接，理论上应该学习到了各个节点的交叉特征，但是从结果上来看，**MLP对这些特征交叉的学习能力确实非常差的**。纠其原因，还是在模型结构的设计上。

![wide&deep模型和deep crossing模型](https://pic1.zhimg.com/80/v2-a76f166f96a59270b7108b352d48e974_720w.jpg)

上图中两种模型，embedding到MLP之间都是做emb-concat，concat的结果表达的交叉信息有限，仅靠MLP想完全捕捉有效交叉信息是困难的，于是在embedding与MLP之间利用更多的数学先验范式做特征交叉是一个改进的方向。

## 1.4 PNN

- 在embedding与MLP之间引入product layer显式地学习每个field的embedding向量间的两两交叉。

![PNN模型框架](https://pic2.zhimg.com/80/v2-6a31278d8d55c8516a891c088c287a79_720w.jpg)

上图中，product layer左边z维embedding层的线性部分，右边p为embedding层的特征交叉部分。思想来源于推荐系统中特征间的交叉关系更多是一种“且”的关系而非“加”的关系。例如，性别为男且喜欢游戏的人群，比起性别男和喜欢游戏的人群，前者的组合比后者更能体现特征交叉的意义。

根据product的方式不同，可以分为inner product（IPNN）和outer product(OPNN)，如下图所示：

![PNN模型的两种不同交叉方式](https://pic3.zhimg.com/80/v2-695f940fca822ff863f6393631fed7ea_720w.jpg)

其中，IPNN模型每个特征是个inner product, f个field两两交叉，得到的新的特征组合有$f*(f-1)/2$个；outer product是两个向量的乘积，得到的新的特征组合有$f*(f-1)/2*k*k$个。

```python

feats = features['feats']  # [batch_size*feat_num, feat_vals]
pairwise_emb = self.pairwise_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim) 
pairwise_emb = tf.reshape(pairwise_emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num, emb_dim)
        
# ----z part. [bs, self.total_feat_num, self.emb_dim] -> [bs, self.total_feat_num*self.emb_dim]
z_part = tf.reshape(pairwise_emb, [-1, self.total_feat_num*self.emb_dim]) 

# IPNN-------------------------
class PairwiseInteraction(BaseModule):
    '特征emb两两元素乘'
    def __init__(self, name, feat_num):
        super(PairwiseInteraction, self).__init__(name)
        self.feat_num = feat_num
        
    @tf.Module.with_name_scope
    def __call__(self, embeddings)
        ''' embeddings: [bs, feat_num, emb_dim]
            return [bs, feat_num*(feat_num-1)/2, emb_dim]
        '''
        elementwise_list = []
        for i in range(self.feat_num):
            for j in range(i+1, self.feat_num):
                elementwise_list.append(tf.multiply(embeddings[:,i,:],embeddings[:,j,:]))  # [(bs,emb_dim),....,] 共feat_num*(feat_num-1)/2个
                
        pairwise_interaction = tf.stack(elementwise_list)  # [feat_num * (feat_num - 1) / 2, bs, emb_dim]
        pairwise_interaction = tf.transpose(pairwise_interaction, perm=[1,0,2])  # [bs, feat_num * (feat_num - 1) / 2, emb_dim]
        return pairwise_interaction  # 交互项矩阵
    
self.p_part_module = PairwiseInteraction(
    name='p_part_module',
    feat_num=self.total_feat_num,
)
p_part = self.p_part_module(pairwise_emb)
p_part = tf.reshape(tf.reduce_sum(p_part, axis=-1), [-1, self.total_pairs])

# OPNN--------------------
if outer_mode == 'mat':
    self.outer_kernel = OuterKernel(
        name='outer_kernel',
        shape=[self.emb_dim,self.total_pairs,self.emb_dim]
    )
elif outer_mode == 'vec':
    self.outer_kernel = OuterKernel(
        name='outer_kernel',
        shape=[self.total_pairs,self.emb_dim]
    )
else: # outer_mode == 'num'
    self.outer_kernel = OuterKernel(
        name='outer_kernel',
        shape=[self.total_pairs,1]
    ) 

if self.product_mode == 'outer':
    p, q = self.pairs_emd_module(pairwise_emb)  # [bs,total_pairs,emb_dim]
    if self.outer_mode == 'mat':
        # p与outer_kernel元素积，结果再与q元素积
        p = tf.expand_dims(p,1)  # [bs, total_pairs, emb_dim] -> [bs,1,total_pairs,emb_dim]
        p_part = tf.reduce_sum(  # [bs, total_pairs]
                 tf.multiply(  # [bs, total_pairs, emb_dim]
                     tf.transpose(  # [bs, total_pairs, emb_dim]
                         tf.reduce_sum(  # [bs, emb_dim, total_pairs]
                             tf.multiply(p, self.outer_kernel),  # [bs,emb_dim,total_pairs,emb_dim]
                             axis=-1
                         ),  # reduce_sum
                         perm=[0,2,1]
                     ),  # transpose
                     q
                 ),  # multiply
                 axis=-1
             )  # reduce_sum
    else:
        # p,q,kernel元素积
        kernel = tf.expand_dims(self.outer_kernel,0)  # vec:[1,total_pairs,emb_dim],num:[1,total_pairs,1] 
        p_part = tf.reduce_sum(p*q*kernel, axis=-1)  # [bs,total_pairs]
# ----deep net
# [bs, self.total_feat_num*self.emb_dim] concat [bs, feat_num*(feat_num-1)/2] -> [bs, self.total_emb_dim+feat_num*(feat_num-1)/2]
deep_input = tf.concat([z_part, p_part], axis=1)
```

## 1.5 DeepFM

- [IJCAI 2017]"DeepFM: A Factorization-Machine based Neural Network for CTR Prediction": https://arxiv.org/abs/1703.04247v1


wide&deep框架虽然强大，但是LR模型需要人工进行特征工程，华为诺亚方舟团队结合FM相比LR的特征交叉，在17年提出了DeepFM，将wide&deep中的LR部分替换为FM，避免人工特征工程，如下图所示：

![deepfm框架](https://pic3.zhimg.com/80/v2-aef79867ec40a8327366333d5764ede2_720w.jpg)

- 更强的低阶特征表达：wide部分取代WDL的LR，与wide&deep模型以及deep crossing模型相比更能捕捉低阶特征信息

- embedding层共享：wide&deep部分的embedding层需要针对deep部分单独设计；而在deepFM中，FM和DEEP部分共享embedding层，FM训练得到的参数既作为wide部分的输出，也作为DNN部分的输入。

- end-end训练：embedding和网络权重联合训练，无需预训练和单独训练

- 缺点：FM部分输出emb_size可能比dnn输出部分小太多

```python
feats = features['feats']  # [batch_size*feat_num, feat_vals]
linear_emb = self.linear_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, 1) 
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1])
second_emb = self.pairwise_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim) 
second_emb = tf.reshape(second_emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num * feat_val_num * emb_dim) feat_val_num=1 in dense

# ----Linear Module
# [bs, feat_num] -> [bs, 1]
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True) 

# ----FM Module
# [bs, feat_num, emb_dim] -> [bs, emb_dim]
# The shape of embedding is [bs, feature_num, emb_dim]
summed_features_emb_square = tf.square(tf.reduce_sum(second_emb,1))  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
squared_sum_features_emb = tf.reduce_sum(tf.square(second_emb),1)  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
FM = 0.5 * tf.subtract(summed_features_emb_square, squared_sum_features_emb)  # [bs, emb_dim] 

# ----Deep Module
# [bs, total_emb_dim] -> [bs, dnn_layers[-1]]
dnn_out = self.dnn_module(tf.reshape(second_emb,shape=[-1, self.total_emb_dim]), is_train)

# ----second order out
# [bs, emb_dim + dnn_layers[-1]] -> [bs, 1]
second_input = tf.concat([FM_out, dnn_out], axis=1)
second_order_out = self.second_order_out(second_input, is_train)  #dense * 1

# ----DeepFM output
global_bias = tf.Variable(name='global_bias', initial_value=tf.zeros_initializer()(shape=[1]), dtype=tf.float32)
global_bias = global_bias * tf.ones_like(global_bias, dtype=tf.float32)  # [bs, 1]  否则报错
model_out = tf.add_n([global_bias, linear_out, second_order_out])  # [bs, 1]
model_out = self.out_act(model_out)
```

## 1.6 NFM(Neural Factorization Machines)

- [SIGIR 2017]"Neural Factorization Machines for Sparse Predictive Analytics": https://arxiv.org/abs/1708.05027
- https://github.com/hexiangnan/neural_factorization_machine

deepFM在embedding层后把FM部分直接**concat**起来（f\*k维，f个field，每个filed是k维向量）作为DNN的输入。Neural Factorization Machines，简称NFM，提出了一种更加简单粗暴的方法，在embedding层后，做了一个叫Bi-interaction的操作，让**各个field做element-wise后sum**起来去做特征交叉，MLP的**输入规模直接压缩到k维**，和特征的原始维度ｎ和特征field维度f没有任何关系，如下图所示：

![NFM模型框架](https://pic1.zhimg.com/80/v2-f38d5cee16581c7af469ba13af7f82a4_720w.jpg)

图中只画出了其中的deep部分, wide部分在这里省略没有画出来。Bi-interaction所做的操作很简单：让**f个field两两element-wise相乘**后，得到f*(f-1)/2个维度为k的向量，然后直接**sum**起来，最后得到一个k维的向量。所以该层**没有任何参数需要学习，同时也降低了网络复杂度，能够加速网络的训练**；但同时这种方法也**可能带来较大的信息损失**。

- 就是把DeepFM中的FM和dnn并联改成了FM和dnn串联

```python
feats = features['feats']  # [batch_size*feat_num, feat_val_num]
linear_emb = self.linear_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, 1) feat_val_num=1 in dense; sparse:(bs*feature_num, 1) because combiner
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1])
emb = self.emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim) feat_val_num=1 in dense; sparse:(bs*feature_num, emb_dize) because combiner
emb = tf.reshape(emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num, emb_dim) feat_val_num=1 in dense; 

# Model
# 原文中此处是x_i为one-hot编码的0或1，样本Xone-hot向量的第i个特征，求加权和
# 官方实现如下
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True)  # [bs, 1]

# ---- Bi_Interaction
# emb shape is [bs,feature_num,emb_dim]
# The shape of embedding is [bs, feature_num, emb_dim]
summed_features_emb_square = tf.square(tf.reduce_sum(second_emb,1))  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
squared_sum_features_emb = tf.reduce_sum(tf.square(second_emb),1)  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
bi_out = 0.5 * tf.subtract(summed_features_emb_square, squared_sum_features_emb)  # [bs, emb_dim] 

# -----Deep
deep_out = self.deep_module(bi_out, is_train=is_train)  # [bs, deep_layer[-1]]
deep_out = self.final_module(deep_out, is_train=is_train)  # [bs, 1]

# NFM output
global_bias = tf.Variable(name='global_bias', initial_value=tf.zeros_initializer(), shape=[1]), dtype=tf.float32)
global_bias = global_bias * tf.ones_like(global_bias, dtype=tf.float32)  # [bs, 1]  否则报错
model_out = tf.add_n([global_bias, linear_out, deep_out])  # [bs, 1] must equal rank
model_out = self.out_act(model_out)
```

## 1.7 AFM

提到的各种网络结构中的FM在做特征交叉时，让不同特征的向量直接做交叉，基于的假设是**各个特征交叉对结果的贡献度是一样的**。这种假设往往不太合理，原因是**不同特征对最终结果的贡献程度一般是不一样的**。Attention Neural Factorization Machines，简称AFM模型，利用了近年来在图像、NLP、语音等领域大获成功的attention机制，在前面讲到的NFM基础上，引入了attention机制来解决这个问题，如下图所示。

![AFM](https://pic1.zhimg.com/80/v2-eb07df13a0010c275bbf8ecba72edb2c_720w.jpg)

AFM的embedding层后和NFM一样，先让f个field的特征做了element-wise product后，得到$f*(f-1)/2$个交叉向量。和NFM直接把这些交叉项sum起来不同，AFM引入了一个Attention Net，认为这些交叉特征项每个对结果的贡献是不同的，例如xi和xj的权重重要度，用aij来表示。从这个角度来看，其实AFM其实就是个**加权累加**的过程。Attention Net部分的权重aij不是直接学习，而是通过如下公式表示:

![AFM公式](https://pic1.zhimg.com/80/v2-941f8604f6e7644f2648410be6da9094_720w.jpg)
![](https://pic4.zhimg.com/80/v2-9bd6536205efe338fc4b1deb182d96c3_720w.png)

其中t表示attention net中的隐层维度，k和前面的一样，为embedding层的维度emb_size。需要学习的参数只有3个，$W,b,h$，参数个数共有$t*k+2*t$个。得到aij权重后，对各个特征两两点积加权累加后，得到一个k维向量，引入一个简单的参数向量pT,维度为k进行学习，和wide部分一起得到最后的AFM输出。

![AFM模型中attention的可视化解释](https://pic3.zhimg.com/80/v2-4b3a096d531539fa495495f2e7d2d7aa_720w.jpg)

关于AFM还有个好处，通过attention-base pooling计算的score值aij体现的是特征vi和vj之间的权重，能够选择有用的二阶特征，如上图所示。

```python

feats = features['feats']  # [batch_size*feat_num, feat_val_num]
linear_emb = self.linear_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, 1)  feat_val_num=1
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1])
pairwise_emb = self.pairwise_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim)  feat_val_num=1 
pairwise_emb = tf.reshape(pairwise_emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num * feat_val_num, emb_dim) feat_val_num=1 

# ----Linear Module
# 官方reduce_sum, [bs, self.total_feat_num] -> [bs, 1]
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True) 

# ----Pairwise Interaction Module
# feat vec两两逐元素相乘 [bs, self.total_feat_num, emb_dim] -> [bs, feat_num * (feat_num - 1) / 2, emb_dim]
elementwise_list = []
for i in range(self.feat_num):
    for j in range(i+1, self.feat_num):
        elementwise_list.append(tf.multiply(pairwise_emb[:,i,:],v[:,j,:]))  # (bs,emb_dim)
        
pairwise_interaction = tf.stack(elementwise_list)  # [feat_num * (feat_num - 1) / 2, bs, emb_dim]
pairwise_interaction_out = tf.transpose(pairwise_interaction, perm=[1,0,2])  # [bs, feat_num * (feat_num - 1) / 2, emb_dim]


# ----Attention Module 公式()第三部分，官方没有h
if self.use_att:
    # relu(wx+b): [bs, feat_num * (feat_num - 1) / 2, emb_dim] -> [bs, feat_num * (feat_num - 1) / 2, att_layer[-1]]
    attention_wx_plus_b = self.attention_nn_module(pairwise_interaction_out, is_train)  # MLP layer
    # P*attention_wx_plus_b: [bs, feat_num * (feat_num - 1) / 2, att_layer[-1]] -> [bs, feat_num * (feat_num - 1) / 2, 1]
    attention_p_mul = tf.reduce_sum(tf.multiply(self.attention_p, attention_wx_plus_b)), 2, keep_dims=True) # att_p is a rand var. 
    # 官方代码增加dropout
    attention_score = tf.nn.dropout(tf.softmax(self.attention_p_mul), self.att_keep_prob)  # [bs, feat_num * (feat_num - 1) / 2, 1]

# ----Attention-aware Pairwise Interaction Layer
if self.use_att:
    # 在FM计算reduce_sum之前对两两特征交叉的结果进行attention
    # att_score * PI: [bs, feat_num * (feat_num - 1) / 2, 1] * [bs, feat_num * (feat_num - 1) / 2, emb_dim] -> [bs, emb_dim]
    afm = tf.reduce_sum(tf.multiply(attention_score, pairwise_interaction_out), axis=1, name='afm')   # [bs, emb_dim]
else:
    # [bs, feat_num * (feat_num - 1) / 2, emb_dim] -> [bs, emb_dim]
    afm = tf.reduce_sum(pairwise_interaction_out, axis=1, name='afm')   # [bs, emb_dim]
    
afm = tf.nn.dropout(afm, self.afm_keep_prob)  # 官方
# second_order output, fc layer
# w matmul afm: [bs, emb_dim] matmul [emb_dim, 1] -> [bs, 1]
second_order_out = self.second_order_out(afm, is_train)

# AFM output
global_bias = tf.Variable(name='global_bias', initial_value=tf.zeros_initializer()(shape=[1]), dtype=tf.float32)
global_bias = global_bias * tf.ones_like(global_bias, dtype=tf.float32)  # [bs, 1]  否则报错 
model_out = tf.add_n([global_bias, linear_out, second_order_out], name='afm_out')  # [bs, 1]  must equal rank
model_out = self.out_act(model_out)
```

## 1.8 DCN

前面提到的几种FM-based的方法都是做的**二阶特征交叉**，如PNN用`product`方式做二阶交叉，NFM和AFM也都采用了`Bi-interaction`的方式学习特征的二阶交叉。对于更高阶的特征交叉，只有让deep去学习了。为解决这个问题，google在2017年提出了Deep&Cross Network，简称DCN的模型，**可以任意组合特征，而且不增加网络参数**。下图为DCN的结构：

![DCN模型](https://pic2.zhimg.com/80/v2-0f89c6100db6fde81b9568046103e509_720w.jpg)

整个网络分4部分组成：

### 1）Embedding and stacking layer

之所以不把`embedding`和`stacking`分开来看，是因为很多时候，`embedding`和`stacking`过程是分不开的。前面讲到的各种 XX-based FM 网络结构，利用FM学到的$v$向量可以很好的作为`embedding`。而在很多实际的业务结构，可能已经有了提取到的`embedding`特征信息，例如图像的特征embedding，text的特征embedding，item的embedding等，还有其他连续值信息，例如年龄，收入水平等，这些embedding向量stack在一起后，一起作为后续网络结构的输入。当然，这部分也可以用前面讲到的FM来做embedding。为了和原始论文保持一致，这里我们假设$X_0$向量维度为$d$（上文的网络结构中为$k$，即emb_size），这一层的做法就是简答的把各种embedding向量concat起来。

![公式](https://pic3.zhimg.com/80/v2-39276f565d2ba8600da6a8334fd82cea_720w.png)

### 2）deep layer network

在embedding and stacking layer之后，网络分成了两路，一路是传统的DNN结构。表示如下：

$$h_{l+1}=f(W_lh_l+b_l)$$

为简化理解，假设每一层网络的参数有$m$个，一共有$L_d$层，输入层由于和上一层连接，有$d*m$个参数（$d$为$X_0$向量维度），后续的$L_{d-1}$层，每层需要$m*(m+1)$个参数，所以一共需要学习的参数有 $d*m+m*(m+1)*(L_{d-1})$。最后的输出也是个$m$维向量

### 3）cross layer network

embedding and stacking layer输入后的另一路就是DCN的重点工作了。每一层$l+1$和前一层$l$的关系可以用如下关系表示:

$$x_{l+1}=x_0x^T_lw_l+b_l+x_l=f(x_l,w_l,b_l)+x_l$$

可以看到f是待拟合的函数，$x_l$即为上一层的网络输入。需要学习的参数为$w_l$和$b_l$，因为$x_l$维度为$d$，当前层网络输入$x_{l+1}$也为$d$维，待学习的参数$w_l$和$b_l$也都是$d$维向量。因此，每一层增加$2*d$的参数（$w$和$b$）需要学习，整体增加$2L_cd$个参数，$L_c$为cross net层数。网络结构如下:

![cross net](https://pic3.zhimg.com/80/v2-a0098c8e0efa55c689b6ededffd94c82_720w.jpg)

经过$L_c$层的`cross layer network`后，在该layer最后一层$L_c$层的输出为$d$维向量

### 4）combination output layer

经过`cross network`的输出$X_{L_1}$($d$维）和`deep network`之后的向量输出（$m$维）直接做`concat`，变为一个$d+m$的向量，最后套一个LR模型，需要学习参数为$1+d+m$。

总结起来，DCN引入的cross network**理论上可以表达任意高阶组合，同时每一层保留低阶组合，参数的向量化也控制了模型的复杂度**。cross网络部分的交叉学习的是**特征向量中每一个element的交叉，本质上是`bit-wise`的**。

```python
# cross net
input_units = x0.shape(1)  #[bs,feat_num * emb_size]
prev = x0
for i in range(cross_layer_num):
    self.kernel = tf.Variable(
               name='kernel',
               initial_value=kernel_initializer(shape=[input_units, 1]),
               dtype=tf.float32
     )
    self.bias = tf.Variable(
              name='bias',
              initial_value=bias_initializer(shape=[input_units]),
              dtype=tf.float32
     )
    xb = tf.matmul(prev, self.kernel)  # matmul([bs, input_units], [input_units,1]) --> [bs, 1]
    prev = x0 * xb + self.bias + prev  # elem_mul([bs, input_units], [bs, 1]) + [input_units,] + [bs, input_units] --> [bs, input_units]
    
```

### 5) cross net部分改进

- DCN-M模型能够简单且有效地建模显式特征交叉，并通过混合低秩矩阵在模型效果和时延上实现了更好的权衡。DCN-M已成功应用于多个大型L2R系统，取得了显著的线下及线上收益。实验结果表明DCN-M的效果超过了现有SOTA方法。

DCN中cross网络的参数是向量，DCN-M中换成了矩阵来提高表达能力、方便落地。DCN-M是指“DCN-matrix” ，原来的DCN在这里称为DCN-V（“DCN-vector”）。

$$x_{l+1}=x_0 \odot (W_lx_l + b_l)+x_l$$

其中$x_l,x_{l+1},b_l \in R^T, W_l \in R^{d*d}$

![dcn-m](../imgs/dcn_m_cross.jpg)

```python
if self.parameterization == 'vector':
    xl_w = torch.tensordot(x_l, self.kernels[i], dims=([1], [0]))  # x' * w (bs, 1)
    dot_ = torch.matmul(x_0, xl_w)  # x0 * (x' * w) (bs, input_units)
    x_l = dot_ + self.bias[i]
elif self.parameterization == 'matrix':
    dot_ = torch.matmul(self.kernels[i], x_l)  # W * xi  (bs, in_features, 1)
    dot_ = dot_ + self.bias[i]  # W * xi + b
    dot_ = x_0 * dot_  # x0 · (W * xi + b)  Hadamard-product 逐元素乘积
x_l = dot_ + x_l
```

### 6） deep和cross的结合方式

结合方式分为堆叠（串行）和并行两种：

![stack_paraller](../imgs/dcn_m_stack_parallel.jpg)

这两种结合方式下的DCN-M效果都优于基准算法。但这两种结构之间的优劣不能一概而论，与数据集有关。串行结构在criteo数据集上更好，而并行结构在Movielen-1M上效果更好。

**DCN-M的损失函数为带L2正则的log loss**

### 7) 低秩优化计算性能

工业界模型往往受计算资源和响应时间限制，需要在保证效果的同时降低计算成本。低秩方法被广泛用于降低计算成本——**将一个稠密矩阵近似分解为两个”高瘦“的低秩矩阵**。而且，当原矩阵的奇异值差异较大或快速衰减时，低秩分解的方法会更加有效。作者发现，DCN-M中学到的参数矩阵是低秩的（所以比较适合做矩阵分解）。下图展示了DCN-M中学到的参数矩阵的奇异值衰减趋势，比初始化的矩阵衰减更快：

![奇异值衰减趋势](../imgs/singular_value.jpg)

因此，作者将参数矩阵$W_l \in R^{d*d}$分解为两个低秩矩阵$U_l,V_l \in R^{d*r}$

公式有两种解释引发了两种改进：

#### （1）在子空间中学习特征交叉

改进1：激发了作者使用Mixture-of-Experts (MoE)的思想，在多个子空间中学习特征交互，然后再进行融合。MOE方法包含两部分：专家网络（即上个公式中使用低秩矩阵分解的cross网络）和门控单元（一个关于输入的函数），通过门控单元来聚合个专家网络的输出结果（综合多个专家的意见，但有可能不采纳某些专家的建议）：

![moe](../imgs/moe.jpg)

#### （2）将输入特征$x$映射到低维空间$R^r$中，然后再映射回到$R^d$

改进2：激发了作者利用映射空间的低秩性。在映射回原有空间之前，施加了非线性变换来提炼特征

此公式的代码实现：（低秩空间中的非线性函数目前采用tanh)

```python
  # E(x_l)
  # project the input x_l to $\mathbb{R}^{r}$
  v_x = torch.matmul(self.V_list[i][expert_id].T, x_l)  # (bs, low_rank, 1)

  # nonlinear activation in low rank space
  v_x = torch.tanh(v_x)
  v_x = torch.matmul(self.C_list[i][expert_id], v_x)
  v_x = torch.tanh(v_x)

  # project back to $\mathbb{R}^{d}$
  uv_x = torch.matmul(self.U_list[i][expert_id], v_x)  # (bs, in_features, 1)

  dot_ = uv_x + self.bias[i]
  dot_ = x_0 * dot_  # Hadamard-product
```

## 1.9 xdeepFM

xDeepFM模型从名字上听好像是deepFM模型的升级，但其实更应该拿来和DCN模型做对比。DCN模型引入了高阶特征交叉，**但是特征的交叉本质上是在bit-wise的**。而**xDeepFM模型认为特征向量$i$和特征向量$j$的交叉过程中，$i$本身的元素交叉没有意义，提取$i$和$j$的向量交叉才是更有效捕捉特征的方式，也就是vector-wise的交叉**，整个模型的框架如下图所示，最核心的模块在于特征交叉的CIN模块。

![xdeepfm结构](https://pic2.zhimg.com/80/v2-b0933b44ea2d81d0e50dac3d14852a99_720w.jpg)

首先我们来看下整个CIN的整体框架图，如下图所示，假设特征field个数是$m$，每个field的隐层维度emb_size为$D$，那么原始embedding层$x_0$的大小为$m*D$，而cross network有$H_k$层，提取的是特征的交叉。每一层网络在做什么事情呢？就是和第一层$x_0$做**特征交叉得到新的特征向量**后，然后这$H_k$层cross net得到的特征向量`concat`到一起，作为MLP的输入。那么，这里面，每一层的特征$x_k$到底是如何输入层$x_0$发生交互的？

![CIN模块结构](https://pic3.zhimg.com/80/v2-5631cee602a80d4042c871abffadea5e_720w.jpg)

以cross net的第$k$层和原始输入$x_0$为例，我们看下如何提取得到新的特征，下图是其特征交叉的过程。其中$x_k$的维度为$H_k*D$，表示的是第$k$层有$H_k$个vector，而原始输入$x_0$的维度为$m*D$，表示输入层有$m$个$D$维的vector。

![CIN模块中特征交叉过程](https://pic3.zhimg.com/80/v2-4722cd10154302bc95c9a5bb021fa6d2_720w.jpg)

![公式](https://pic2.zhimg.com/80/v2-8d14066673a531c347975c572828a1b1_720w.png)

这里$W^{k,h}$表示的是第$k$层的第$h$个vector的权重，是模型需要学习的参数。整个公式的理解是整个xdeepFM理解的关键，我们具体看下发生了什么

（1） 首先，从前一层的输入$X_{k-1}$(一共有$H_{k-1}$个vector)，取出任意一个vector；从原始输入$x_0$(一共有$m$个vector),取出任意一个vector，【两者两两做哈达码积(对应元素相乘)】（两个向量矩阵乘法，见代码），可以得到$H_{k-1}*m$个vector

（2） 这$H_{k-1}*m$个交叉得到的vector，每个vector维度都是$D$，我们通过一个$W$矩阵做乘积进行加权求和，相当于是个带权重的pooling, 最终得到加权求和后的vector $X_h^k$，表示的是第$k$层第$h$个vector。这里的$W$矩阵就是模型要学习的

（3） 为什么说是压缩，压缩体现在哪里？还是用图说话，这里我们看下原始论文给出的图示，有助于整个过程的理解。

![CIN模块具体结构](https://pic3.zhimg.com/80/v2-1e356d090454af1ab555cb27a49fe936_720w.jpg)

在上图的左图中，我们把$D$看成是原始二维平面的宽度，我们沿着$D$的方向挨个进行计算。先看$x_k$向量中$D$的第一维，有$H_k$个数；$x_0$向量中$D$的第一维，有$m$个数，让$H_k$和$m$两两计算，就可以得到$H_k*m$的一个平面。一直沿着$D$方向，2，3，4,…D，我们就可以得到一个$H_k*m*D$的三维矩阵，暂且叫做$z_{k+1}$，注意这个过程只是简单的矩阵计算，没有参数需要学习。

在右边的图中，我们开始提取前面$z_{k+1}$的信息，还是以$D$方向的第一维为例，一个$m*H_k$的平面，乘以一个大小一样的$m*H_k$矩阵$W$，然后加权求和，就可以得到这个平面最后压缩的一个实数。整个平面被“压缩“成为了一个一维的数。一直沿着D方向求解每个平面压缩后的数，就可以得到一个$D$维的向量。

这就是整个“压缩”的取名原因。整个过程非常类似CNN的filter卷积思想，$W$就是卷积核，得到的每个特征映射的值就是feature map。

```python
# https://github.com/NELSONZHAO/zhihu/blob/master/ctr_models/xDeepFM.ipynb
def compressed_interaction_net(x0, xl, D, n_filters):
    """
    求l+1层的featuremap
    @param x0: 原始输入
    @param xl: 第l层的输入
    @param D: embedding dim,也有用k表示的
    @param n_filters: 压缩网络filter的数量，l+1增的特征数量H_{l+1}
    """
    # 这里设x0中共有H_0 = m个特征，xl中共有H_l = h个特征
    
    # 1.将x0与xl按照k所在的维度（-1）进行拆分，每个都可以拆成k列
    x0_cols = tf.split(x0, D, axis=-1)  # ?, m, D
    xl_cols = tf.split(xl, D, axis=-1)  # ?, h, D
    
    assert len(x0_cols)==len(xl_cols), print("error shape!")
    
    # 2.遍历k列，对于x0与xl所在的第i列进行外积计算，存在feature_maps中
    feature_maps = []
    for i in range(D):
        feature_map = tf.matmul(xl_cols[i], x0_cols[i], transpose_b=True)  # 外积 ?, h, m
        feature_map = tf.expand_dims(feature_map, axis=-1)  # ?, h, m, 1
        feature_maps.append(feature_map)
    
    # 3.得到 h × m × D 的三维tensor
    feature_maps = Concatenate(axis=-1)(feature_maps)  # ?, h, m, D
    
    # 3.压缩网络
    x0_n_feats = x0.get_shape()[1]  # m
    xl_n_feats = xl.get_shape()[1]  # h
    reshaped_feature_maps = Reshape(target_shape=(x0_n_feats * xl_n_feats, D))(feature_maps)  # ?, h*m, D
    transposed_feature_maps = tf.transpose(reshaped_feature_maps, [0, 2, 1])  # ?, D, h*m
    
    new_feature_maps = Conv1D(n_filters, 1, 1)(transposed_feature_maps)  # ?, D, n_filters
    new_feature_maps = tf.transpose(new_feature_maps, [0, 2, 1])  # ?, n_filters, D
    
    return new_feature_maps
```

## 1.10 FGCNN

CNN模型在图像，语音，NLP领域都是非常重要的特征提取器，原因是对图像、视频、语言来说，存在着**很强的local connection信息**。而在推荐系统中由于特征输入都是稀疏无序的，很难直接利用CNN作为特征提取。华为诺亚方舟在2019年的WWW会议上提出了一种巧妙的利用CNN提取特征的方法FGCNN，整个模型框架如下图所示：

![FGCNN](https://pic4.zhimg.com/80/v2-5fbab9ecc6a01339a6204f63bec59ee3_720w.jpg)

其中利用CNN提取新特征是整个paper的核心模块，具体框架如下图所示，可以分为4个层：卷积层、池化层、重组层、特征输出层。下面分别介绍这4个不同的层，分别看下各自的输入，输出以及具体的网络结构。

![CNN模块从原始特征提取新特征](https://pic4.zhimg.com/80/v2-a6819039d7afebbce4aa7100137a042b_720w.jpg)

### 1.10.1 conv层

![conv的特征卷积过程](https://pic1.zhimg.com/80/v2-5fa903a7fab4be1d6247b07260baeec0_720w.jpg)

原始one-hot特征经过embedding后，进入的是卷积层。卷积的思想和TextCNN类似，通过一个高度为hp，宽度为d的卷积核进行卷积；其中高度hp代表的是**每次卷积连接的邻域的个数**，以上图为例，$hp=2$，表示**二维特征的交叉**，**从上到下卷积**，表示的是N&A, A&H, H&G卷积，分别表示名字和年龄、年龄和身高、身高和性别的交叉；而宽度方向d需要和embedding的维度d保持一致，这和TextCNN的思想是一致的，**只在高度方向上（特征种类上）进行卷积**。

1）Convolutional layer 输入

特征one-hot之后经过embedding层后的输出作为卷积层的输入，输入维度为$n_f*k$。$n_f$为field个数或者理解为feature_num，也是高度，$k$为隐层维度

2）Convolutional layer 输出

经过二维平面(channel=1)大小(指宽k和高nf)不变，增加第三维卷积核(channel)大小，第1层卷积层的输出大小为$n_f*k*m_c^1$，$m_c^1$为第$1$个卷积层的卷积核个数，以$l=1$为例，$C_{:,:,i}$表示的是第1层卷积层第$i$个feature-map，每个feature map的大小为$n_f*k$，$C_{p,q,i}$表示的是第一层第i个feature map的坐标为（p, q）的元素:

![](https://pic1.zhimg.com/80/v2-a59885a520d8fe166d176fa6f21a883c_720w.jpg)

### 1.10.2 pooling层

特征经过第一层卷积层之后，从$n_f*k$变成了$n_f*k*m_c^1$，维度反而扩了$m_c^1$倍，这和预期的特征降维不一致，解决办法和CNN里常用的套路一样，通过pooling做降维。该paper使用的是`max pooling`，在原先卷积高度$n_f$上选择最大值作为卷积后的特征表达，表达如下所示:

![pool](https://pic1.zhimg.com/80/v2-a08c0070215dec2af484ec259c16972c_720w.png)

1) pooling层输入

卷积层的输出作为池化层的输入，以第1层为例，pooling层的输入维度为$n_f*k*m_c^1$

2) pooling层输出

卷积层输出的时候在**高度方向上选择max，只有高度（特征数量）方向维度发生变化**，卷积输出后维度为$n_f/h_p * k *m_c^1$，**$h_p$为卷积核高度**，相当于沿着field的方向压缩了$h_p$倍

### 1.10.3 Recombination层

经过特征卷积层和特征池化层后，可以通过特征重组Recombination layer生成新的特征组了。还是以第1层的Recombination layer为例，$m_c^1$为原始feature map的个数，$m_r^1$为新生成的特征feature map的个数:

![](https://pic4.zhimg.com/80/v2-7705b0912b39df43c4f2a7df4fd4da0f_720w.jpg)
![](https://pic4.zhimg.com/80/v2-eb9a283096100f5e374c191a1dfe5b7b_720w.jpg)


其中$s^1$为池化层的输出，$R^1$为重组层的输出，而$WR^1$和$BR^1$为该层待学习的参数

1）Recombination层输入

卷积层的输出$S^1$为该层的输入，通过$WR^1$的矩阵进行特征的recombination

2）Recombination层输出

新生成的特征维度为$n_f/h_p*k*m_r^1$

### 1.10.4 特征输出层Concatenation Layer

第1层`Convolution layer->pooling layer->recombination layer`后，生成第1层的新特征$R^1$，连同第2，3，4，…，nc层的新特征R2，R3，R4，…，Rnc层一起concat起来，组成新的特征R=(R1，R2，…，Rnc)。之后**R可以和原始特征一样做各种可能的交叉**，例如和原始特征做二阶交叉等，再一起输入到模型层:

![FGCNN新特征的生成](https://pic1.zhimg.com/80/v2-a969e57f2c0758c8d52a55d0002b1410_720w.jpg)

![](https://pic4.zhimg.com/80/v2-dddf14c0584166877327c7aae3c58427_720w.png)

1）concatenation层输入

重组层Recombination layer输出的新特征R1, R2, R3, $R^{n_c}$，和embedding层输出的原始特征作为concatenation layer的输入

2）concatenation层输出

原有的特征E和新特征R连在一起作为concatenation的特征输出，作为NN层的输入

$E \in R^{n_f * k * 1}, R \in R^{}$

```python

for i in range(1,len(self.filters)+1):
    self.conv_layers.append(tf.keras.layers.Conv2D(filters=filters, kernel_size=(width, 1), strides=(1, 1),
                                                   padding='same',
                                                   activation='tanh', use_bias=True, ))
    self.pooling_layers.append(
        tf.keras.layers.MaxPooling2D(pool_size=(pooling_width, 1)))
    
    # 重组，pool后输出的h*w保持不变，只变channle为指定数量
    self.dense_layers.append(tf.keras.layers.Dense(pooling_shape[1] * embedding_size * new_filters, 
                                    activation='tanh', use_bias=True))
self.flatten = tf.keras.layers.Flatten()
# ......
# --------------------
# ......
embedding_size = int(inputs.shape[-1])
pooling_result = tf.expand_dims(inputs, axis=3)

new_feature_list = []
for i in range(1, len(self.filters) + 1):
    new_filters = self.new_maps[i - 1]  # 获取当前需要的filter数量

    conv_result = self.conv_layers[i - 1](pooling_result)  # H*W不变，改变channel为指定filter数量即特征数量

    pooling_result = self.pooling_layers[i - 1](conv_result)  # 特征数量压缩，其他不变

    flatten_result = self.flatten(pooling_result)  # 拉平，方便后续进行重组

    new_result = self.dense_layers[i - 1](flatten_result)  # 重组只改变channel数量

    new_feature_list.append(
        tf.reshape(new_result, (-1, int(pooling_result.shape[1]) * new_filters, embedding_size)))

new_features = concat_func(new_feature_list, axis=1)  # 新特征concat到一起 (bs,?,embedding_size)

# 之后
combined_input = concat_func([origin_input, new_features], axis=1)
inner_product = tf.keras.layers.Flatten()(InnerProductLayer()(  # 拉平的新旧特征两两配对进行逐元素对应相乘，即IPNN中的操作
    tf.keras.layers.Lambda(unstack, mask=[None] * int(combined_input.shape[1]))(combined_input)))
linear_signal = tf.keras.layers.Flatten()(combined_input)
dnn_input = tf.keras.layers.Concatenate()([linear_signal, inner_product])
dnn_input = tf.keras.layers.Flatten()(dnn_input)

final_logit = DNN(dnn_hidden_units, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout)(dnn_input)
```

## 1.11 AutoInt模型（Auto Feature Interaction）

## 1.12 DIN（Deep Interest Network）

工业界的做法不像学术界，很多模型网络结构优化并不一味的追求模型的复杂和网络结构有多fancy，每一步背后都有大量的业务思考后沉淀下来的。阿里这篇din也如此。在了解din之前，我们先看下din前身的模型，GwEN模型（Group-wise Embedding Network，阿里内部称呼）

![GwEN模型结构（DIN的baseline）](https://pic3.zhimg.com/80/v2-2700f25d520621c4d017154b1a610cba_720w.jpg)

前面讲到的很多模型，输入层都是大规模稀疏特征，经过embedding层后输入到MLP网络中。这里的一个假设就是，每个field都是one-hot的，如果不是one-hot而是multi-hot，那么就用pooling的方式，如sum pooling，average pooling，max pooling等，这样才能保证每个特征field embedding都是定长的。DIN的前身GwEN模型也一样，对于multi-hot特征的典型代表，用户历史行为，比如用户在电商系统里购买过的商品，往往都是几十几百甚至几千的，需要经过sum pooling和其他特征concat一起。【也可以特征+特征值构成字符串一起hash，然后设定最大长度截断补零】

而这种数学假设其实往往都是和实际的发生场景不一致的。例如一个女性用户过去在淘宝买过白色针织衫、连衣裙、帽子、高跟鞋、单肩包、洗漱用品等，当前候选商品是一件黑色外套，白色针织衫对黑色外套的权重影响应该更大，洗漱用品权重应该更小。**如果将这些历史行为过的商品做sum pooling，那么无论对于当前推荐什么商品，用户都是一个固定向量的表达，信息损失很大，显然优化空间很大**。

![din](https://pic3.zhimg.com/80/v2-5a7a190efa328860983f79d4567b7ac2_720w.jpg)

针对sum/average pooling的缺点，DIN提出了一种`local activation`的思想，基于一种基本的假设：**用户历史不同的行为，对当前不同的商品权重是不一样的**。例如用户过去有a, b, c三个行为，如果当前商品是d,那么a, b, c的权重可能是0.8，0.2，0.2；如果是商品e，那么a, b, c的权重可能变成了0.4，0.8，0.1。也就是说，**不同的query，激发用户历史行为不同的keys的权重是不一样的**。

![](https://pic2.zhimg.com/80/v2-15db337ce1b5963d9ecd8646d518dd6d_720w.jpeg)

(1）query: 用户历史行为，长度为$H$，$e_1，e_2，…，e_H$表示用户历史行为的向量表示

(2）keys: 当前候选广告（店铺、类目、或者其他item实体）

关于DIN里的`activation weight`还有个可以稍微讲几句的点。两个向量的相似度，在各种CF的方法的时候基本是用的点积或者cosine，2017年DIN挂在arXiv的版本中是使用了两个向量本身以及concat后进入MLP得到其相似度，2018发在KDD的版本中多了`outer product`，以及`向量相减`，相当于引入和保留了**更多特征的信息**。另外作者在文章提到为了保持不同历史行为对当前attention的影响，**权重也不做归一化**，这个和原始的attention也有所不同。

![不同版本att](https://pic4.zhimg.com/80/v2-9b922273eef6b9f9c0f02bcde2b0f653_720w.jpg)

作为工业界的落地实践，阿里在DIN上很“克制”的只用了最能表达用户个性化需求的特征——**用户行为keys**，而**query也是当前候选的商品广告**，与线上提升ctr的指标更为吻合，对工业界的推荐系统来说借鉴意义还是很大的。当然这不是说之前的其他attention机制模型没用，不同的数据集，不同的落地场景需求不一致，也给工业界更多的尝试提供了很多思路的借鉴。

## 1.13 DIEN(Deep Interest Evolution Network)

在前面讲到的模型中，所使用的特征都是**时间无序**的，din也如此，用户的行为特征之间并没有先后顺序，**强调的是用户兴趣的多样性**。但是实际**用户的兴趣应该是在不断进化的，用户越近期的行为，对于预测后续的行为越重要，而用户越早期的行为，对于预测后续行为的权重影响应该小一点**。因此，为了**捕获用户行为兴趣随时间如何发展变化**，在din提出一年后，阿里又进一步提出了DIEN，引入了时间序列概念，深度兴趣进化网络。

DIEN文章里提到，在以往的推荐模型中存在的序列模型中，主要利用RNN来捕获用户行为序列也就是用户历史行为数据中的依赖关系，比对用户行为序列直接做pooling要好。但是以往这些模型有两个缺点:

- 直接将RNN的隐层作为兴趣表达，而一般隐层的表达和真正表达的商品embedding一般不是等价的，并不能直接反映用户的兴趣；
- RNN将用户历史行为的每个行为看成*等权*的一般来说也不合理。

整个DIEN的整体框架，如下图所示：

![DIEN模型框架](https://pic3.zhimg.com/80/v2-23a102a3f50d5a9497f20a3c96254cd6_720w.jpg)

### (1) 输入层

和DIN的输入一样。按照类型可以分为4大类

- 用户画像特征：如年龄、性别、职业等

- context特征：如网络环境、时间、IP地址、手机型号等，与user以及item无关

- target ad特征：当前候选广告

- 用户行为特征：DIEN里最重要的能体现用户个性化的特征，对于每个用户来说，假设有T个历史行为，按照发生的先后顺序依次输入模型

### (2) embedding层

将one-hot特征转为dense的embedding向量

### (3) 兴趣抽取层（interest extractor layer）

该层的主要作用和DIN一样，为了从embedding层中抽取出用户的兴趣。该paper认为**用户当前对候选广告是否感兴趣，是和历史行为behavior有关的**，所以**引入了GRU的序列模型来拟合抽取用户兴趣**。

![](https://pic2.zhimg.com/80/v2-7af023cb9a5cceacbde8290b17791425_720w.jpg)

经过GRU结构后，商品的embedding表达从$e(t)$变成了$h(t)$，表示第$t$个行为序列的embedding表达。

![DIEN中的辅助loss结构](https://pic1.zhimg.com/80/v2-871302b6ec88eab2af873d651827c8ec_720w.jpg)

除了GRU结构提取隐层的向量，DIEN还引入了有监督学习，**强行让原始的行为向量$e(t)$和$h(t)$产生交互**。如上图所示，引入了辅助loss(auxiliary loss)，当前时刻$h(t)$作为输入，下一刻的输入$e(t+1)$认为是正样本(click)，负样本进行**负采样**（不等于当前时刻）；然后让$h(t)$与正负样本**分别**做向量内积，辅助loss定义为：

![](https://pic2.zhimg.com/80/v2-c80230d8a80f4228df1d89eaf97699cd_720w.jpg)

最终loss表示为：
![](https://pic1.zhimg.com/80/v2-828fa7b6cb9dbbc43ccb43e843757304_720w.png)

其中$α$为超参数，代表的是辅助loss对整体loss的贡献。有了这个辅助loss，$t$时刻提取的隐层向量$h(t)$可以比原始的$h(t)$更有助于表达用户兴趣，也可以**加速网络的训练过程**。

### (4) 兴趣进化层（interest evolving layer）

理论上来说，$h(t)$如果替代$e(t)$作为商品的最终表达其实也是可以的，把用户序列$t=1,2,3,…,T$当成用户的$T$个行为过的商品，然后和当前的候选广告套用DIN的attention 网络去计算每个行为和当前候选广告的权重，最终得到用户的历史行为加权表达也是完全ok的。但作者认为**用户的行为模式是会发展的**，因此引入了第二层GRU网络来**学习每个历史行为序列和当前候选广告之间的权重**.

对于每个历史行为$h_t$，当前候选广告$e_a$，通过`softmax`求出两者的权重。注意这里不是直接向量点击，而是引入了矩阵$W$，可以认为是简单的一层*全连接网络*:

![](https://pic1.zhimg.com/80/v2-ea28b2a27bc91a45ed91342881d1fb54_720w.jpg)

如何使用这里学习的attention作为兴趣进化层的输入，作者又提出了三种计算方法

1) AIGRU(attention input with GRU)

最基础的版本，兴趣进化层第$t$个行为序列的input就是隐层$h_t$的加权

![](https://pic1.zhimg.com/80/v2-caf3347fbd5f3d03caf438e581a04c7c_720w.png)

作者尝试后发现效果并不好，原因是如果是输入0，也会参与到隐层$h_t$的计算和更新，相当于给用户兴趣的提取引入了噪音，不相关的用户行为会干扰模型的学习

2) AGRU（attention base GRU）

这里作者使用了attention权重$a_t$来取代原始GRU中的更新门，表达如下

![](https://pic4.zhimg.com/80/v2-6bfe3ad144148b5cc5ee58ca69fdfed3_720w.png)

3) AUGRU（GRU with attentional update gate）

这里作者依然对原始GRU做了改造，公式如下:

![](https://pic4.zhimg.com/80/v2-b8f9d5c22b6ac2bb7243eac1bf7776d7_720w.jpg)

![AUGRU结构](https://pic3.zhimg.com/80/v2-f1927fad0eb3baa962216716bbe1cb92_720w.jpg)

其中，$u_t'$引入了$a_t$来取代原有的更新向量$u_t$，表达的是当前$u_t$对结果的影响。如果当前权重$a_t$较大，$u_t'$也较大，当前时刻$h_t'$保留更多，上一个时刻$h_{t-1}$影响也会少一点。从而整个AUGRU结果可以更平滑的学习用户兴趣

## 1.14 DSIN(Deep Session Interest Network)

- https://mp.weixin.qq.com/s/kGWiRH6ntSmNTLhAG49AbQ
- https://github.com/shenweichen/DeepCTR/blob/e9c8f08f861293d5dc2f4e005b7b6ed47125ef17/deepctr/models/dsin.py#L146

主要贡献在于对**用户的历史点击行为划分为不同session,对每个session使用Transformer学习session embedding，最后使用BiLSTM对session序列建模**。

这篇文章和阿里的另一篇DIEN(  Deep Interest Evolution Network  )很像，「主要区别在于用户历史行为序列的刻画，**引入了session，并用transformer学习session表征**」 。

![DSIN](../imgs/dsin.jpg)

模型主要由四个部分组成:

- Session Division Layer: 对用户的历史行为划分到不同session
- Session Interest Extractor Layer: 学习session的表征
- Session Interest Interacting Layer: 学习session之间的演变
- Session Interest Activating Layer: 学习当前item和历史点击session的相关性


### 1.14.1 Session Division Layer

- 将用户的历史点击行为序列划分为多个sessions。

假设: **相同session用户的行为是接近的**，比如都是在点一些和裤子相关的商品，而**不同session间的用户行为是有差异的**，比如后面可能是点一些和洗发水相关的商品。

文章将间隔超过30分钟作为session的划分，比如将历史点击序列$S$转换为session $Q$，第    $k$个session表示为：

$$Q_k = [b_1,...,b_i,...,b_T] \in R^{T*d_{model}}$$

其中$T$是第$k$个session的长度，$d_{model}$是输入item的embeddin大小，即emb_size

### 1.14.2 Session Interest Extractor Layer

- 获取session表征。

用户在相同session内的行为是高度相关的，此外用户在session内的一些随意行为会偏离整个session表达。

`Bias Encoding`: **刻画不同session间的顺序**

`multi-head self-attention`**对每个session建模**: **刻画相同session内行为间的相关性，同时减小不相关行为的影响**

1) Bias Encoding

为了刻画不同session的顺序关系，使用bias encoding，定义如下：

$$BE(k,t,c)=w^K_k+w^T_t+w^C_c$$

其中$BE(k,t,c) \in R^{k*t*d_{model}}$表示第$k$个session中第$t$个物品的embedding向量的第$c$个位置的偏置项。即对每个session中的每个物品对应的embedding的每个位置，都加入了偏置项。加入bias encoding之后，用户的session表示为$Q=Q+BE$

2) Multi-head Self-attention

在推荐系统中，用户的点击行为会受各种因素影响，比如颜色、款式和价格。`Multi-head self attention`可以在不同的子空间上建模这种关系。这里将第$k$个session划分为 $H$个$head$  ，使用 $Q_k=[Q_{k1},...,Q_{kh},...,Q_{kH}]$表示，其中$Q_{kh} \in R^{T*d_h},d_h=\frac{1}{H}d_{model}$.$head_h$可以表示为：

$head_h=Attention(Q_{kh}W^Q,Q_{kh}W^K,Q_{kh}W^V)=softmax(\frac{Q_{kh}W^QW^{K^T}Q^T_{kh}}{\sqrt {d_{model}}})Q_{kh}W^V$

其中，$W^Q,W^K,W^V$是线性矩阵。

然后将$head$的输出concat到一起接入FC网络：

$I^Q_k=FFN(Concat(head_1,..,head_H)W^O)$

其中$FFN(·)$表示fc网络，$W^O$表示线性矩阵

经过 `Multi-head self attention` 处理之后，每个session得到的结果仍然是 $T*d_{model}$大小，之后经过一个`avg pooling`操作，将每个session兴趣转换为一个$d_{model}$维向量 ：

$I_k=Avg(I^Q_k)$

### 1.14.3 Session Interest Interacting Layer

使用双向LSTM建模session之间的演变。$H_t=h_{ft}\oplus h_{bt}$

上式中的两项分别表示前向的隐藏层状态和反向的隐藏层状态。

### 1.14.4 Session Interest Activating Layer

这部分主要是通过Attention机制刻画Item和session之间的相关性。用户的session与目标物品越相近，越应该赋予更大的权重。使用注意力机制来刻画这种相关性：

$a^I_k=\frac{exp(I_kW^IX^I)}{\sum ^K_k(I_KW^IX^I)}$

$U^I=\sum ^{K}_{k}\alpha ^I_kI_k$

其中$X^I$是目标item的embedding向量。

从实验结果来看，AUC相比DIEN都有 「0.5%」 以上的提升。

这篇文章在我们自己的业务场景(新闻推荐场景)上也实验了，选取了用户最近的10次点击session，session长度最大取20，使用7天训练，1天测试，最后离线AUC相比DIEN大概有「0.46%」的提升，和文章的结果基本吻合。个人感觉这篇文章的设计还是比较巧妙，充分利用了用户行为的内在结构，使用session划分用户的点击序列，使得在使用LSTM等序列模型对历史点击建模的时候，长度大大缩小，而且效果还有所提升。